In [1]:
import cmdstanpy
import numpy as np
import pandas as pd

from itertools import product
cmdstanpy.install_cmdstan();

CmdStan install directory: /Users/igor.michels/.cmdstan
CmdStan version 2.34.0 already installed
Test model compilation


In [2]:
def generate_matchups(n_equipes, habilidades, home_force):
    if home_force is None: home_force = 0
    confrontos = []
    for equipe1, equipe2 in product(range(1, n_equipes + 1), repeat=2):
        if equipe1 == equipe2: continue
        prob = np.exp(habilidades[equipe1 - 1] + home_force)
        prob = prob / (np.exp(habilidades[equipe1 - 1] + home_force) + np.exp(habilidades[equipe2 - 1]))
        confrontos.append({
            'equipe1': equipe1,
            'equipe2': equipe2,
            'vitoria_equipe1': (np.random.random() < prob) * 1
        })

    df_confrontos = pd.DataFrame(confrontos)

    return df_confrontos

def generate_seasons(n_equipes, habilidades, n_seasons, home_force=None):
    df = pd.DataFrame()
    for _ in range(n_seasons):
        df = pd.concat([df, generate_matchups(n_equipes, habilidades, home_force)])
    
    return df

In [3]:
num_seasons = 5
num_equipes = 20
habilidades = np.random.normal(0, 1, num_equipes)
df = generate_seasons(num_equipes, habilidades, num_seasons)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/model.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': num_equipes,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[1:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[1:]))
sum(inside_ci) / num_equipes

11:13:06 - cmdstanpy - INFO - compiling stan file /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model.stan to exe file /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model
11:13:10 - cmdstanpy - INFO - compiled model executable: /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model
11:13:11 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

11:13:22 - cmdstanpy - INFO - CmdStan done processing.


0.95

In [4]:
num_seasons = 10
num_equipes = 20
home_force = abs(np.random.normal(0, 1))
habilidades = np.random.normal(0, 1, num_equipes)
df = generate_seasons(num_equipes, habilidades, num_seasons, home_force)

modelo_compilado = cmdstanpy.CmdStanModel(stan_file='../models/model_2.stan', force_compile=False)
dados_modelo = {
    'num_jogos': len(df),
    'num_equipes': num_equipes,
    'equipe1': df['equipe1'].values,
    'equipe2': df['equipe2'].values,
    'vitoria_equipe1': df['vitoria_equipe1'].values,
}

ajuste = modelo_compilado.sample(data=dados_modelo, chains=4, iter_warmup=2000, iter_sampling=10000, show_console=False)
habilidades += ajuste.summary()['Mean']['habilidade[1]'] - habilidades[0]
inside_ci = (habilidades > ajuste.summary()['5%'].values[2:])
inside_ci *= ((habilidades < ajuste.summary()['95%'].values[2:]))
sum(inside_ci) / num_equipes

11:13:24 - cmdstanpy - INFO - compiling stan file /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model_2.stan to exe file /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model_2
11:13:28 - cmdstanpy - INFO - compiled model executable: /Users/igor.michels/Documents/FGV/BrazilianSoccerEDA/Modeling/models/model_2
11:13:28 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

11:14:11 - cmdstanpy - INFO - CmdStan done processing.
11:14:11 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 433 divergent transitions (4.3%)
	Chain 2 had 362 divergent transitions (3.6%)
	Chain 3 had 446 divergent transitions (4.5%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


1.0

In [5]:
home_force

0.7274480028937723

In [6]:
ajuste.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,8.450680e+06,3.572610e+06,5.797180e+06,148796.0000,1.033850e+07,1.644050e+07,2.63308,0.031038,3.18428
home_force,-1.184680e+01,2.379330e+01,3.365650e+01,-70.6512,7.583700e+00,8.066090e+00,2.00091,0.023586,88.00730
habilidade[1],-2.453140e+01,3.040610e+01,4.300940e+01,-99.8478,5.534280e-01,1.646220e+00,2.00081,0.023585,90.97300
habilidade[2],-2.686670e+01,3.272040e+01,4.628260e+01,-107.8320,-4.236550e-01,1.644350e+00,2.00078,0.023585,99.14940
habilidade[3],-3.076870e+01,3.724510e+01,5.268230e+01,-122.8510,-1.061960e-01,6.532010e-04,2.00074,0.023584,113.02600
habilidade[4],-2.581470e+01,3.274590e+01,4.631880e+01,-106.8450,1.886390e+00,2.070760e+00,2.00078,0.023585,97.52820
habilidade[5],-3.743200e+01,4.537430e+01,6.417950e+01,-149.2120,-6.764570e-01,7.919420e-01,2.00065,0.023583,186.32000
habilidade[6],-2.282080e+01,2.825540e+01,3.996750e+01,-92.8612,6.526330e-02,8.800750e-01,2.00083,0.023586,86.30150
habilidade[7],-3.395150e+01,4.246920e+01,6.007020e+01,-138.5610,1.285180e+00,1.934780e+00,2.00065,0.023583,185.93500
habilidade[8],-2.940580e+01,3.657600e+01,5.173580e+01,-119.8290,4.531020e-01,7.225180e-01,2.00074,0.023584,113.30000
